In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import os

In [ ]:
data_path = 'D:\\Data\\text_summarization\\'

In [ ]:
with open(data_path + 'hvg_article_urls.txt', 'r') as f:
    articles = [a.strip() for a in f.readlines()]

In [ ]:
if os.path.isfile(data_path + 'hvg_bad_article_urls.txt'):
    with open(data_path + 'hvg_bad_article_urls.txt', 'r') as f:
        bad_article_urls = set([a.strip() for a in f.readlines()])
else:
    bad_article_urls = set()
    
len(bad_article_urls)

In [ ]:
def get_content(article):
    title, description, tags, published_time, modified_time, section, content = [None]*7
    
    for meta in article.find_all('meta'):
        meta_content = meta.get('content')
        name = meta.get('name')
        itemprop = meta.get('itemprop')
        property = meta.get('property')
        
        if itemprop == 'name' and title is None:
            title = meta_content
        if property == 'og:title':
            title = meta_content
        
        if itemprop == 'description' and description is None:
            description = meta_content
        if property == 'og:description':
            description = meta_content
         
        if itemprop == 'datePublished' and published_time is None:
            published_time = meta_content
        if itemprop == 'dateModified' and modified_time is None:
            modified_time = meta_content
        if property == 'article:published_time':
            published_time = meta_content
        if property == 'article:modified_time':
            modified_time = meta_content
            
        if property == 'article:tag':
            tags = meta_content

        if property == 'article:section':
            section = meta_content
            
    content = str(article.findAll("div", {"class": "entry-content"})[-1])
    
    return title.strip(), description.strip(), tags.strip(), published_time, modified_time, section.strip(), content.strip()

In [ ]:
def get_article(url):
    filename = '_'.join(url.split('/')[3:])
    path = data_path + 'hvg/' + filename + '.txt'
    
    if os.path.isfile(path) or url in bad_article_urls:
        return
    
    article_content = requests.get(url).content.decode('utf-8','ignore')
    article = BeautifulSoup(article_content, 'html.parser')
    processed = get_content(article)

    with open(path, 'w', encoding="utf-8") as f:
        for data in processed:
            f.write(data + '\n')

In [ ]:
for article_url in tqdm(articles):
    try:
        get_article(article_url)
    except Exception as e:
#         print(article_url)
        with open(data_path + 'hvg_bad_article_urls.txt', 'a') as f:
            f.write(article_url + '\n')